In [1]:
import os
import IPython
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

In [2]:
mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [3]:
zip_path = tf.keras.utils.get_file(
    origin='https://itsci.mju.ac.th/downloads/watcharin/datasets/pv/cmmju_15min_filled_extra.csv.tar.gz')
print(zip_path)
df = pd.read_csv(zip_path, compression='tar')

C:\Users\ASUS\.keras\datasets\cmmju_15min_filled_extra.csv.tar.gz


In [4]:
df.head(3)

,datetime,ambient_temp,current_power,value_of_consumption,external_energy_supply,grid_feed_in,internal_power_supply,module_temp,total_irradiation,cc,...,tp,u10,v10,equation_of_time_min,zenith_deg,elevation_deg,azimuth_deg,declination_deg,hour_angle_deg,incidence_deg
0,2021-11-06 11:15:00,30.288889,46519.355556,37280.422222,0.0,9238.933333,37280.422222,58.022222,798.444444,0.000002,...,0.0,0.0,0.0,16.244239,36.998137,53.001863,158.619243,-15.751553,-13.176451,36.998137
1,2021-11-06 11:30:00,30.076923,47600.714286,39472.626374,0.0,9397.756098,39132.406593,58.263736,812.571429,0.000002,...,0.0,0.0,0.0,16.243603,35.874705,54.125295,164.395912,-15.754719,-9.426610,35.874705
2,2021-11-06 11:45:00,30.629213,47056.988764,32669.393258,0.0,14387.595506,32669.393258,57.235955,807.224719,0.000001,...,0.0,0.0,0.0,16.242965,35.105017,54.894983,170.471336,-15.757885,-5.676769,35.105017


In [5]:
for col in df.columns:
    missing_count = df[col].isnull().sum()
    if missing_count > 0:
        print(f"{col}: {missing_count} missing values")

incidence_deg: 69011 missing values


In [6]:
df.pop('value_of_consumption')
df.pop('external_energy_supply')
df.pop('grid_feed_in')
df.pop('internal_power_supply')
df.head(2)

,datetime,ambient_temp,current_power,module_temp,total_irradiation,cc,q,r,t,u,...,tp,u10,v10,equation_of_time_min,zenith_deg,elevation_deg,azimuth_deg,declination_deg,hour_angle_deg,incidence_deg
0,2021-11-06 11:15:00,30.288889,46519.355556,58.022222,798.444444,0.000002,0.011270,64.187104,293.434588,0.544700,...,0.0,0.0,0.0,16.244239,36.998137,53.001863,158.619243,-15.751553,-13.176451,36.998137
1,2021-11-06 11:30:00,30.076923,47600.714286,58.263736,812.571429,0.000002,0.011312,64.567093,293.392390,0.529152,...,0.0,0.0,0.0,16.243603,35.874705,54.125295,164.395912,-15.754719,-9.426610,35.874705


In [7]:
# Check what years exist in your dataset
date_col = 'datetime'  # Replace with your actual column name

df = df.sort_values(date_col).reset_index(drop=True)

# fill the NaN value with zero
df = df.fillna(0)

# Ensure timestamp column is datetime type
if df[date_col].dtype != 'datetime64[ns]':
    df[date_col] = pd.to_datetime(df[date_col])

# Sort by time if it exists
df.index = df[date_col]
df.pop(date_col)
df = df.sort_index()
print(df.shape)

df.head(5)

(138238, 29)


,ambient_temp,current_power,module_temp,total_irradiation,cc,q,r,t,u,v,...,tp,u10,v10,equation_of_time_min,zenith_deg,elevation_deg,azimuth_deg,declination_deg,hour_angle_deg,incidence_deg
datetime,,,,,,,,,,,,,,,,,,,,,
2021-11-06 11:15:00,30.288889,46519.355556,58.022222,798.444444,2.297362e-06,0.011270,64.187104,293.434588,0.544700,-1.063590,...,0.0,0.0,0.0,16.244239,36.998137,53.001863,158.619243,-15.751553,-13.176451,36.998137
2021-11-06 11:30:00,30.076923,47600.714286,58.263736,812.571429,2.376975e-06,0.011312,64.567093,293.392390,0.529152,-0.912537,...,0.0,0.0,0.0,16.243603,35.874705,54.125295,164.395912,-15.754719,-9.426610,35.874705
2021-11-06 11:45:00,30.629213,47056.988764,57.235955,807.224719,1.268101e-06,0.011341,64.877710,293.356146,0.512290,-0.752512,...,0.0,0.0,0.0,16.242965,35.105017,54.894983,170.471336,-15.757885,-5.676769,35.105017
2021-11-06 12:00:00,31.021978,48415.835165,56.846154,817.527473,0.000000e+00,0.011366,65.189689,293.320821,0.497185,-0.620988,...,0.0,0.0,0.0,16.242326,34.712629,55.287371,176.742301,-15.761050,-1.926929,34.712629
2021-11-06 12:15:00,31.831461,49345.404494,55.842697,816.651685,-6.155763e-07,0.011393,65.556480,293.282514,0.484595,-0.543996,...,0.0,0.0,0.0,16.241685,34.710348,55.289652,183.081860,-15.764215,1.822911,34.710348


In [8]:
for col in df.columns:
    missing_count = df[col].isnull().sum()
    if missing_count > 0:
        print(f"{col}: {missing_count} missing values")

In [ ]:
plot_cols = ['module_temperature', 'current_power', 'total_irradiation']
plot_features = df[plot_cols]
plot_features.index = df.index
_ = plot_features.plot(subplots=True)

plot_features = df[plot_cols][:480]
plot_features.index = df.index[:480]
_ = plot_features.plot(subplots=True)

In [ ]:
df.describe().transpose()

In [ ]:
print(df.index.year.unique())
print(df.index.year.value_counts())

In [ ]:
df_proced = df.copy()
df_proced = df_proced.ffill() # forward fill to impute missing values
df_proced.head(3)

In [ ]:
df_proced['hour'] = df_proced.index.hour
df_proced['year'] = df_proced.index.year
df_proced['month'] = df_proced.index.month
df_proced['day_of_month'] = df_proced.index.day
df_proced['day_of_week'] = df_proced.index.dayofweek  # 0=Monday, 6=Sunday
df_proced['is_weekend'] = (df_proced['day_of_week'] >= 5).astype(int)

df_proced['season'] = df_proced['month'] % 12 // 3 + 1  # 1=Winter, 2=Spring, 3=Summer, 4=Fall
df_proced['is_daytime'] = ((df_proced['hour'] >= 6) & (df_proced['hour'] <= 18)).astype(int)
        

In [ ]:
available_mapping = [col for col in df_proced.columns if col in lag_mapping]
available_mapping

In [ ]:
# สร้าง lag 1 ชั่วโมง
for orig_col in available_mapping:
    df_proced[f'{orig_col}_Lag1'] = df_proced[orig_col].shift(1)
    print(f"  ✅ สร้าง {orig_col}_Lag1 จาก {orig_col}")

In [ ]:
# สร้าง lag 24 ชั่วโมง
for orig_col in available_mapping:
    df_proced[f'{orig_col}_Lag24'] = df_proced[orig_col].shift(24)
    print(f"  ✅ สร้าง {orig_col}_Lag24 จาก {orig_col}")

In [ ]:
# ลบแถวแรกๆ ที่มีค่า NaN จากการสร้าง lag
initial_rows = len(df_proced)
df_proced = df_proced.dropna()
rows_removed = initial_rows - len(df_proced)

print(f"⚠️  ลบแถวที่มีค่า NaN จาก lag features: {rows_removed} แถว")
print(f"📊 จำนวนข้อมูลหลังเพิ่ม lag features: {len(df_proced)} แถว")

df_proced.head(3)

In [ ]:
mean = df_proced.mean()
std = df_proced.std()

df_proced = (df_proced - mean) / std

# Shift to non-negative values using Min-Max scaling
min_val = df_proced.min()
max_val = df_proced.max()
df_proced = (df_proced - min_val) / (max_val - min_val)

In [ ]:
#df_std = (df - train_mean) / train_std
df_std = df_proced.melt(var_name='Column', value_name='Normalized')
plt.figure(figsize=(12, 6))
ax = sns.violinplot(x='Column', y='Normalized', data=df_std)
ax.set_xticks(range(len(df_proced.keys())))
ax.set_xticklabels(df_proced.keys(), rotation=90)

In [ ]:
print(df_proced.index.min())
print(df_proced.index.max())

In [ ]:
column_indices = {name: i for i, name in enumerate(df_proced.columns)}

n = len(df_proced)
test_df = df_proced[0:int(n*0.2)]
train_df = df_proced[int(n*0.2):int(n*0.8)]
val_df = df_proced[int(n*0.8):]

num_features = df_proced.shape[1]

print(f'Training set: {train_df.shape}')
print(f'Validation set: {val_df.shape}')
print(f'Test set: {test_df.shape}')

In [ ]:
print(f'Column indices: {column_indices}')
print(f"num_features: {num_features}")

In [ ]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in enumerate(label_columns)}
    self.column_indices = {name: i for i, name in enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def split_window(self, features):
    inputs = features[:, self.input_slice, :]
    labels = features[:, self.labels_slice, :]
    if self.label_columns is not None:
      labels = tf.stack([labels[:, :, self.column_indices[name]] for name in self.label_columns], axis=-1)

    # Slicing doesn't preserve static shape information, so set the shapes
    # manually. This way the `tf.data.Datasets` are easier to inspect.
    inputs.set_shape([None, self.input_width, None])
    labels.set_shape([None, self.label_width, None])

    return inputs, labels

  def plot(self, model=None, plot_col='current_power', max_subplots=3):
    inputs, labels = self.example
    plt.figure(figsize=(12, 8))
    plot_col_index = self.column_indices[plot_col]
    max_n = min(max_subplots, len(inputs))
    for n in range(max_n):
      plt.subplot(max_n, 1, n+1)
      plt.ylabel(f'{plot_col} [normed]')
      # plot input value
      plt.plot(self.input_indices, inputs[n, :, plot_col_index], label='Inputs', marker='.', zorder=-10)

      if self.label_columns:
        label_col_index = self.label_columns_indices.get(plot_col, None)
      else:
        label_col_index = plot_col_index

      if label_col_index is None:
        continue

      # plot label
      plt.plot(self.label_indices, labels[n, :, label_col_index], label='Labels', c='#2ca02c', marker='.', zorder=-9)
        
      if model is not None:
        predictions = model(inputs)
        plt.plot(self.label_indices, predictions[n, :, label_col_index], label='Predictions', c='#ff7f0e', marker='.', zorder=-8)

      if n == 0:
        plt.legend()

    plt.xlabel('Time [15min]')

  def make_dataset(self, data):
    data = np.array(data, dtype=np.float32)
    ds = tf.keras.utils.timeseries_dataset_from_array(
        data=data,
        targets=None,
        sequence_length=self.total_window_size,
        sequence_stride=1,
        shuffle=True,
        batch_size=32,)

    ds = ds.map(self.split_window)

    return ds
    
  @property
  def train(self):
    return self.make_dataset(self.train_df)

  @property
  def val(self):
    return self.make_dataset(self.val_df)

  @property
  def test(self):
    return self.make_dataset(self.test_df)

  @property
  def example(self):
    """Get and cache an example batch of `inputs, labels` for plotting."""
    result = getattr(self, '_example', None)
    if result is None:
      # No example batch was found, so get one from the `.train` dataset
      result = next(iter(self.train))
      # And cache it for next time
      self._example = result
    return result
  
  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [ ]:
wide_window = WindowGenerator(
    input_width=96, label_width=96, shift=1,
    label_columns=['current_power'])

wide_window

In [ ]:
# Stack three slices, the length of the total window.
example_window = tf.stack([np.array(train_df[:wide_window.total_window_size]),
                           np.array(train_df[100:100+wide_window.total_window_size]),
                           np.array(train_df[200:200+wide_window.total_window_size])])

example_inputs, example_labels = wide_window.split_window(example_window)

print('All shapes are: (batch, time, features)')
print(f'Window shape: {example_window.shape}')
print(f'Inputs shape: {example_inputs.shape}')
print(f'Labels shape: {example_labels.shape}')

In [ ]:
wide_window.plot()

In [ ]:
# Each element is an (inputs, label) pair.
wide_window.train.element_spec

In [ ]:
for example_inputs, example_labels in wide_window.train.take(1):
  print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
  print(f'Labels shape (batch, time, features): {example_labels.shape}')

In [ ]:
single_step_window = WindowGenerator(
    input_width=1, label_width=1, shift=1,
    label_columns=['current_power'])
single_step_window

In [ ]:
for example_inputs, example_labels in single_step_window.train.take(1):
  print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
  print(f'Labels shape (batch, time, features): {example_labels.shape}')

In [ ]:
class Baseline(tf.keras.Model):
  def __init__(self, label_index=None):
    super().__init__()
    self.label_index = label_index

  def call(self, inputs):
    if self.label_index is None:
      return inputs
    result = inputs[:, :, self.label_index]
    return result[:, :, tf.newaxis]

In [ ]:
baseline = Baseline(label_index=column_indices['current_power'])

baseline.compile(loss=tf.keras.losses.MeanSquaredError(),
                 metrics=[tf.keras.metrics.MeanAbsoluteError()])

val_performance = {}
performance = {}
val_performance['1-Baseline'] = baseline.evaluate(single_step_window.val, return_dict=True)
performance['1-Baseline'] = baseline.evaluate(single_step_window.test, return_dict=True)
wide_window.plot(baseline)

In [ ]:
linear = tf.keras.Sequential([
    tf.keras.layers.Dense(units=1)
])

In [ ]:
print('Input shape:', wide_window.example[0].shape)
print('Output shape:', linear(wide_window.example[0]).shape)

In [ ]:
MAX_EPOCHS = 50
LEARNING_RATE = 0.001

#models_dir = 'src/jupyter_nb/experiments/lag_models'
models_dir = 'lag_models'

def fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')
  
  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

def load_or_train_model(model_name, model, window, is_custom=False):
  """
  Load model if it exists, otherwise train and save it.
  
  Args:
      model_name: Name of the model (e.g., 'linear_model')
      model: The Keras model object
      window: Training window
      is_custom: True for custom Model subclasses (saves weights only)
  
  Returns:
      Trained model
  """
  # Add appropriate extension based on model type
  if is_custom:
    model_filename = f"{model_name}.weights.h5"
  else:
    model_filename = f"{model_name}.keras"
  
  model_path = os.path.join(models_dir, model_filename)
  
  model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Nadam(learning_rate=LEARNING_RATE),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])
  
  if os.path.exists(model_path):
    print(f"✓ Loading existing model: {model_name}")
    if is_custom:
      # For custom models, load weights only
      model.load_weights(model_path)
    else:
      # Use safe_mode=False to allow loading models with serializable layers
      model = tf.keras.models.load_model(model_path, safe_mode=False)
    return model
  else:
    print(f"→ Training model: {model_name}")
    history = fit(model, window)
    print(f"✓ Training complete: {model_name}")

    if not os.path.exists(models_dir):
        os.makedirs(models_dir)
        print(f"Created directory: {models_dir}")

    if is_custom:
      # For custom models, save weights only
      model.save_weights(model_path)
    else:
      model.save(model_path)
    
    print(f"✓ Model saved to {model_path}")
    return model

In [ ]:
linear = load_or_train_model('linear_model', linear, wide_window)

val_performance['2-Linear'] = linear.evaluate(wide_window.val, return_dict=True)
performance['2-Linear'] = linear.evaluate(wide_window.test, return_dict=True)
wide_window.plot(linear)

In [ ]:
print('Input shape:', wide_window.example[0].shape)
print('Output shape:', linear(wide_window.example[0]).shape)

In [ ]:
plt.bar(x = range(len(train_df.columns)),
        height=linear.layers[0].kernel[:,0].numpy())
axis = plt.gca()
axis.set_xticks(range(len(train_df.columns)))
_ = axis.set_xticklabels(train_df.columns, rotation=90)

In [ ]:
dense = tf.keras.Sequential([
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(units=1)
])

dense = load_or_train_model('dense_model', dense, wide_window)

val_performance['3-Dense'] = dense.evaluate(wide_window.val, return_dict=True)
performance['3-Dense'] = dense.evaluate(wide_window.test, return_dict=True)
wide_window.plot(dense)

In [ ]:
OUT_STEPS = 96
INPUT_STEPS = 672
multi_window = WindowGenerator(input_width=INPUT_STEPS,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               label_columns=['current_power'])

multi_window.plot()

In [ ]:
class MultiStepLastBaseline(tf.keras.Model):
  def call(self, inputs):
    return tf.tile(inputs[:, -1:, :], [1, OUT_STEPS, 1])

last_baseline = MultiStepLastBaseline()
last_baseline.compile(loss=tf.keras.losses.MeanSquaredError(),
                      metrics=[tf.keras.metrics.MeanAbsoluteError()])

val_performance['A-Last'] = last_baseline.evaluate(multi_window.val, return_dict=True)
performance['A-Last'] = last_baseline.evaluate(multi_window.test, return_dict=True)
multi_window.plot(last_baseline)

In [ ]:
CONV_WIDTH = 3
multi_conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Cropping1D(cropping=(INPUT_STEPS-CONV_WIDTH, 0)),
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(128, activation='relu', kernel_size=(CONV_WIDTH)),
    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

multi_conv_model = load_or_train_model('multi_conv_model', multi_conv_model, multi_window)

IPython.display.clear_output()

val_performance['B-Conv'] = multi_conv_model.evaluate(multi_window.val, return_dict=True)
performance['B-Conv'] = multi_conv_model.evaluate(multi_window.test, return_dict=True)
multi_window.plot(multi_conv_model)

In [ ]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units].
    # Adding more `lstm_units` just overfits more quickly.
    tf.keras.layers.LSTM(128, return_sequences=False),
    # Shape => [batch, out_steps*features].
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features].
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

multi_lstm_model = load_or_train_model('multi_lstm_model', multi_lstm_model, multi_window)

IPython.display.clear_output()

val_performance['C-LSTM'] = multi_lstm_model.evaluate(multi_window.val, return_dict=True)
performance['C-LSTM'] = multi_lstm_model.evaluate(multi_window.test, return_dict=True)
multi_window.plot(multi_lstm_model)

In [ ]:
class FeedBack(tf.keras.Model):
  def __init__(self, units, out_steps):
    super().__init__()
    self.out_steps = out_steps
    self.units = units
    self.lstm_cell = tf.keras.layers.LSTMCell(units)
    # Also wrap the LSTMCell in an RNN to simplify the `warmup` method.
    self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
    self.dense = tf.keras.layers.Dense(num_features)

  def warmup(self, inputs):
    # inputs.shape => (batch, time, features)
    # x.shape => (batch, lstm_units)
    x, *state = self.lstm_rnn(inputs)

    # predictions.shape => (batch, features)
    prediction = self.dense(x)
    return prediction, state
  
  def call(self, inputs, training=None):
    # Use a TensorArray to capture dynamically unrolled outputs.
    predictions = []
    # Initialize the LSTM state.
    prediction, state = self.warmup(inputs)

    # Insert the first prediction.
    predictions.append(prediction)

    # Run the rest of the prediction steps.
    for n in range(1, self.out_steps):
        # Use the last prediction as input.
        x = prediction
        # Execute one lstm step.
        x, state = self.lstm_cell(x, states=state, training=training)
        
        # Convert the lstm output to a prediction.
        prediction = self.dense(x)
        # Add the prediction to the output.
        predictions.append(prediction)

    # predictions.shape => (time, batch, features)
    predictions = tf.stack(predictions)
    # predictions.shape => (batch, time, features)
    predictions = tf.transpose(predictions, [1, 0, 2])
    return predictions

In [ ]:
feedback_model = FeedBack(units=128, out_steps=OUT_STEPS)

In [ ]:
prediction, state = feedback_model.warmup(multi_window.example[0])
prediction.shape

In [ ]:
print('Output shape (batch, time, features): ', feedback_model(multi_window.example[0]).shape)

In [ ]:
feedback_model = load_or_train_model('feedback_model', feedback_model, multi_window, is_custom=True)

IPython.display.clear_output()

val_performance['D-AR LSTM'] = feedback_model.evaluate(multi_window.val, return_dict=True)
performance['D-AR LSTM'] = feedback_model.evaluate(multi_window.test, return_dict=True)
multi_window.plot(feedback_model)

In [ ]:
x = np.arange(len(performance))
width = 0.3

metric_name = 'mean_absolute_error'
val_mae = [v[metric_name] for v in val_performance.values()]
test_mae = [v[metric_name] for v in performance.values()]

plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=performance.keys(), rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()

In [ ]:
print('Validation performance:')
for name, value in val_performance.items():
  print(f'{name:8s}: {value[metric_name]:0.4f}')

In [ ]:
print('\nTest performance:')
for name, value in performance.items():
  print(f'{name:8s}: {value[metric_name]:0.4f}')